In [1]:
import streaminghub_datamux as dm
import numpy as np
import fastplotlib as fpl

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Available devices:
🯄 (default) | Apple M2 Pro | IntegratedGPU | Metal | 


In [2]:
# constants
dataset = "ADHD_SIN"
timeout = 10
screen_wh = (1920, 1080)
diag_dist = (21, 22)
freq = 60

# hyperparameters
vt = 10

In [3]:
mode = "remote"

# setup datamux api
if mode == "local":
    api = dm.API()

elif mode == "remote":
    api = dm.RemoteAPI("websocket", "msgpack")
    api.connect("full-darling-gar.ngrok-free.app", 80)

else:
    raise ValueError(mode)

Loaded streaminghub_datamux.rpc.client:websocket
Loaded streaminghub_datamux.rpc.codec:msgpack


In [4]:
streams = api.list_collection_streams(dataset)  # for recorded data (ADHD_SIN)
# streams = api.list_live_streams("pupil_core")  # for live data (pupil_core)

# get the first stream
eye_stream = streams[2]
print(eye_stream)

name='eye movements' description='' unit='pixel, mm' frequency=60 fields={'lx': Field(name='left x', description='', dtype=<class 'numpy.float32'>), 'ly': Field(name='left y', description='', dtype=<class 'numpy.float32'>), 'ld': Field(name='left d', description='', dtype=<class 'numpy.float32'>), 'rx': Field(name='right x', description='', dtype=<class 'numpy.float32'>), 'ry': Field(name='right y', description='', dtype=<class 'numpy.float32'>), 'rd': Field(name='right d', description='', dtype=<class 'numpy.float32'>)} index={'t': Field(name='timestamp', description='', dtype=<class 'numpy.float32'>)} node=Node(id='ADHD_SIN', device=None, uri=None, inputs={}, outputs={}) attrs={'subject': '012', 'noise': '10', 'task': '13', 'collection': 'ADHD_SIN', 'id': 'eye', 'mode': 'replay'}


In [5]:
# define a transform to map data into (t,x,y,d) format and handle missing values
merge = dm.ExpressionMap(
    {
        "t": "t",
        "x": "(lx + rx) / 2",
        "y": "(ly + ry) / 2",
        "d": "(ld + rd) / 2",
    }
)

cast = dm.ExpressionMap(
    {
        "t": "float(t)",
        "x": "float(x)",
        "y": "float(y)",
        "d": "float(d)",
    }
)

In [6]:
class LogWriter(dm.SinkTask):

    def __init__(self) -> None:
        super().__init__()

    def step(self, input) -> int | None:
        print(input)

In [7]:
class GazePlot2D(dm.SinkTask):

    def __init__(self) -> None:
        super().__init__(mode="thread")
        # make all-zeros data point
        self.n = 100
        self.buffer = np.full((self.n, 3), np.nan, dtype=np.float32)
        self.running = True
        
        # figure with 1 rows and 1 column
        shape = (1,1)
        controller_ids = [[0]]
        
        # create the figure
        self.fig = fpl.Figure(shape=shape, cameras='2d', controller_ids=controller_ids)
        
        for i, subplot in enumerate(self.fig):
            # create and add the LineGraphic
            subplot.add_line(data=self.buffer, thickness=3, cmap='jet', name="xy")
            # make axes visible
            subplot.set_axes_visibility(False)
            subplot.set_grid_visibility(True)
        display(self.fig.show())
        self.fig.add_animations(lambda: self.update_view())

    # a function to move the ball along the buffer
    def update_view(self):
        if not self.running:
            return
        for subplot in self.fig:
            subplot["xy"].data[:] = self.buffer[:]
            subplot.auto_scale()

    def step(self, input) -> int | None:
        item = np.array([[input['x'], input['y'], input['t']]])
        self.buffer = np.concatenate( (self.buffer[-self.n+1:], item), axis=0)

In [8]:
# define pipeline
pipeline_A = dm.Pipeline(
    api.attach(eye_stream, transform=merge).with_name("eye"),
    dm.Filter("not (isnan(x) or isnan(y) or isnan(d))"),
    dm.Transform(cast),
    GazePlot2D().with_name("2d_gaze_plot"),
).with_name("simple_logger")
# run pipeline
pipeline_A.run(timeout, block=False)

RFBOutputContext()

Attempting to remove object that was not a child.


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

[15:12:28] INFO     pipeline completed                         ]8;id=954462;file:///Users/yasith/projects/streaminghub/streaminghub_datamux/src/streaminghub_datamux/transforms.py\transforms.py]8;;\:]8;id=969702;file:///Users/yasith/projects/streaminghub/streaminghub_datamux/src/streaminghub_datamux/transforms.py#187\187]8;;\
